# 深度学习3.1：线性回归

### 理论
1. 模型原理
   
   设 $X$ 为特征，$y$ 为标签，预测值 $\hat{y}$ 由 $wX + b$ 给出
2. 损失函数
   
   衡量预测值与真实值的差异，这里采用 $L(w,b) = \frac{1}{n} \sum_{i=1}^{n}\frac{1}{2}(wx_i+b-y_i)^2 $
3. 显式解（解析解）
   
   略

### 随机梯度下降（SGD）
解决寻找合适的参数 $\mathbf{w}$ 的问题，也就是数值解。（合适的参数是指损失函数最小）

1. 原理
   
   - 假设样本为 $B$，选择一个小样本 $b$ 和初始参数 $\mathbf{w_0}$，在 $b$ 上计算损失函数，然后计算梯度。
   - 引入超参数 $\eta$，表示步长，即在下一次的更新中，参数 $\mathbf{w_0} = (w_0,w_1...w_n,b)$ 会沿着负梯度方向走 $\eta$ “步”。这就完成了一次更新。
   - 引入超参数 $T$，表示迭代次数。也就是总共需要走的“步”的次数。
2. 超参数
   
   即训练过程中需要人为设定的参数。

### 代码实现的尝试

- 接下来是我的尝试（从零实现）：

In [97]:
# 模拟数据
import torch as t

# 参数
u = 0.0
w = 0.0
W = t.tensor([w,u]).reshape(2,1)
# 数据集，最好都用浮点数
# 第二列的1是偏置项系数
a = t.tensor([1.,1])
b = t.tensor([4.,1])
c = t.tensor([2.,1])
d = t.tensor([6.,1])
x = t.stack([a,b,c,d])
y = t.tensor([4.99,17.08,8.97,25.03]).reshape(-1,1) # 列向量
# 超参数
eta = 0.001
T = 1000
'OK'

'OK'

In [98]:
# 梯度函数
def gradient(W,x,y):
    # 参数
    y_hat = x @ W
    y_hat = y_hat.reshape(4,1)
    return t.tensor([t.sum((y_hat - y)*x[:,0]), t.sum(y_hat - y)])
gradient(W,x,y)

tensor([-728.9100,  -56.0700])

In [99]:
# SGD 实现
# 调参
eta = 0.001
T = 1000
res = t.tensor([w,u])
for i in range(T):
    res = res - gradient(res,x,y)*eta
res

tensor([4.2134, 0.3241])

- 接下来是 DeepSeek 修改后的代码：

In [118]:
# 初始化参数（直接使用Tensor，支持自动求导）
'''
V = t.tensor([0.0, 0.0], requires_grad=True).reshape(2, 1)  ← 这是错误的！因为对 V 进行了reshape，V就不再是叶子节点了！没有 grad 属性！
'''
V = t.tensor([[0.0], [0.0]], requires_grad=True)
# 超参数
eta = 0.01  # 调整学习率
T = 2000     # 增加迭代次数

# 训练循环
for epoch in range(T):
    # 前向计算
    y_hat = x @ V
    # 计算损失（均方误差）
    loss = ((y_hat - y) ** 2).mean()
    # 反向传播（自动计算梯度）
    loss.backward()
    # 手动更新参数（关闭梯度跟踪）
    with t.no_grad():
        V -= eta * V.grad
        V.grad.zero_()  # 清零梯度

print(V)

tensor([[4.0137],
        [0.9729]], requires_grad=True)


- 反思：
     - 梯度计算未取平均，数值过大导致步长变大。
     - 使用自动求导时，要注意叶子节点的属性。